In [1]:
%pip install mediapipe opencv-python --user
%pip install opencv-python-headless
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install mediapipe opencv-python

In [3]:
# MEDIAPIPE POSE ESTIMATION SETUP

import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import os
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from IPython.display import display, HTML

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=True,  # Set to False for video
    model_complexity=2,      # Higher accuracy
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

I0000 00:00:1741270696.886746 2568791 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3


In [4]:
def process_image(image_path):
    """Process an image and extract pose landmarks"""
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    return image, image_rgb, results

def visualize_pose(image_rgb, results):
    """Visualize detected pose landmarks on an image"""
    annotated_image = image_rgb.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS
    )
    plt.figure(figsize=(10, 10))
    plt.imshow(annotated_image)
    plt.title("Detected Pose")
    plt.axis('off')
    plt.show()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
#BODY PART MAPPING

# Define constants for pose comparison
POSE_LANDMARKS = {
    'left_wrist': mp_pose.PoseLandmark.LEFT_WRIST.value,
    'right_wrist': mp_pose.PoseLandmark.RIGHT_WRIST.value,
    'left_elbow': mp_pose.PoseLandmark.LEFT_ELBOW.value,
    'right_elbow': mp_pose.PoseLandmark.RIGHT_ELBOW.value,
    'left_shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER.value,
    'right_shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
    'left_hip': mp_pose.PoseLandmark.LEFT_HIP.value,
    'right_hip': mp_pose.PoseLandmark.RIGHT_HIP.value,
    'left_knee': mp_pose.PoseLandmark.LEFT_KNEE.value,
    'right_knee': mp_pose.PoseLandmark.RIGHT_KNEE.value,
    'left_ankle': mp_pose.PoseLandmark.LEFT_ANKLE.value,
    'right_ankle': mp_pose.PoseLandmark.RIGHT_ANKLE.value,
    'left_foot_index': mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
    'right_foot_index': mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
}

# Define groups for specific analysis
ARM_LANDMARKS = [
    'left_wrist', 'right_wrist', 'left_elbow', 'right_elbow', 
    'left_shoulder', 'right_shoulder'
]

FOOT_LANDMARKS = [
    'left_hip', 'right_hip', 'left_knee', 'right_knee',
    'left_ankle', 'right_ankle', 'left_foot_index', 'right_foot_index'
]

In [6]:
# REFERENCE POSE MANAGER CLASS
class ReferencePoseManager:
    """Manages reference poses for Spanish dance movements"""
    
    def __init__(self, data_dir="spanish_dance_references"):
        """Initialize the reference pose manager"""
        self.data_dir = data_dir
        self.reference_poses = {}
        self.ensure_data_dir()
        
    def ensure_data_dir(self):
        """Ensure the data directory exists"""
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)
            
    def add_reference_pose(self, pose_name, image_path, pose_type="arm", description=""):
        """Add a reference pose to the database"""
        image, _, results = process_image(image_path)
        
        if not results.pose_landmarks:
            print(f"No pose detected in {image_path}")
            return False
        
        # Extract landmarks as normalized vectors
        landmarks = []
        for landmark in results.pose_landmarks.landmark:
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])
        
        self.reference_poses[pose_name] = {
            'landmarks': landmarks,
            'type': pose_type,  # 'arm' or 'foot'
            'description': description,
            'image_path': image_path
        }
        
        # Save to disk
        self.save_references()
        return True
    
    def save_references(self):
        """Save reference poses to disk"""
        with open(os.path.join(self.data_dir, "reference_poses.pkl"), "wb") as f:
            pickle.dump(self.reference_poses, f)
    
    def load_references(self):
        """Load reference poses from disk"""
        ref_file = os.path.join(self.data_dir, "reference_poses.pkl")
        if os.path.exists(ref_file):
            with open(ref_file, "rb") as f:
                self.reference_poses = pickle.load(f)
    
    def visualize_reference_pose(self, pose_name):
        """Visualize a reference pose"""
        if pose_name not in self.reference_poses:
            print(f"Reference pose '{pose_name}' not found")
            return
        
        ref_data = self.reference_poses[pose_name]
        image, image_rgb, results = process_image(ref_data['image_path'])
        
        print(f"Reference Pose: {pose_name}")
        print(f"Type: {ref_data['type']}")
        print(f"Description: {ref_data['description']}")
        
        visualize_pose(image_rgb, results)


In [7]:
# POSE ANALYZER CLASS
class PoseAnalyzer:
    """Analyzes dancer poses against reference poses"""
    
    def __init__(self, reference_manager):
        """Initialize the pose analyzer"""
        self.ref_manager = reference_manager
        
    def normalize_pose(self, landmarks):
        """Normalize pose by centering and scaling"""
        # Extract x, y coordinates
        coords = np.array([[lm.x, lm.y] for lm in landmarks])
        
        # Find the bounding box
        min_x, min_y = np.min(coords, axis=0)
        max_x, max_y = np.max(coords, axis=0)
        
        # Scale and center
        scale_x = 1.0 / (max_x - min_x) if max_x > min_x else 1.0
        scale_y = 1.0 / (max_y - min_y) if max_y > min_y else 1.0
        
        normalized = []
        for lm in landmarks:
            normalized_lm = mp_pose.PoseLandmark()
            normalized_lm.x = (lm.x - min_x) * scale_x
            normalized_lm.y = (lm.y - min_y) * scale_y
            normalized_lm.z = lm.z  # Keep z as is
            normalized_lm.visibility = lm.visibility
            normalized.append(normalized_lm)
        
        return normalized
    
    def extract_landmark_vectors(self, landmarks, landmark_group):
        """Extract vectors for specific landmark groups (arm or foot)"""
        vectors = []
        
        if landmark_group == "arm":
            target_landmarks = ARM_LANDMARKS
        elif landmark_group == "foot":
            target_landmarks = FOOT_LANDMARKS
        else:
            # Use all landmarks
            return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten().reshape(1, -1)
        
        # Extract coordinates for target landmarks
        coords = []
        for name in target_landmarks:
            idx = POSE_LANDMARKS[name]
            lm = landmarks[idx]
            coords.extend([lm.x, lm.y, lm.z])
        
        return np.array(coords).reshape(1, -1)
    
    def compare_poses(self, dancer_landmarks, ref_pose_name, view="front"):
        """Compare dancer pose to a reference pose"""
        if ref_pose_name not in self.ref_manager.reference_poses:
            return {"score": 0, "message": f"Reference pose '{ref_pose_name}' not found"}
        
        ref_data = self.ref_manager.reference_poses[ref_pose_name]
        
        # Determine which landmark group to use based on view and pose type
        landmark_group = None
        if view == "front" and ref_data['type'] == 'arm':
            landmark_group = "arm"
        elif view == "side" and ref_data['type'] == 'foot':
            landmark_group = "foot"
        
        # Convert reference landmarks format
        ref_landmarks = []
        for lm_data in ref_data['landmarks']:
            lm = mp_pose.PoseLandmark()
            lm.x, lm.y, lm.z, lm.visibility = lm_data
            ref_landmarks.append(lm)
        
        # Normalize both poses
        norm_dancer = self.normalize_pose(dancer_landmarks)
        norm_ref = self.normalize_pose(ref_landmarks)
        
        # Extract vectors for comparison
        dancer_vector = self.extract_landmark_vectors(norm_dancer, landmark_group)
        ref_vector = self.extract_landmark_vectors(norm_ref, landmark_group)
        
        # Calculate similarity score (cosine similarity)
        similarity = cosine_similarity(dancer_vector, ref_vector)[0][0]
        
        # Scale to 0-100%
        score = max(0, min(100, (similarity + 1) * 50))
        
        # Generate feedback message
        message = self._generate_feedback(score, ref_pose_name, ref_data)
        
        return {
            "score": score,
            "message": message
        }
    
    def _generate_feedback(self, score, pose_name, ref_data):
        """Generate feedback based on score"""
        if score >= 90:
            return f"Excellent execution of {pose_name}! {ref_data['description']}"
        elif score >= 75:
            return f"Good execution of {pose_name}. Minor adjustments needed. {ref_data['description']}"
        elif score >= 60:
            return f"Fair execution of {pose_name}. Review the position details. {ref_data['description']}"
        else:
            return f"Needs improvement on {pose_name}. Focus on the key elements: {ref_data['description']}"
    
    def analyze_dance_sequence(self, dancer_image_path, reference_sequence, view="front"):
        """Analyze a dancer's execution of a sequence of poses"""
        dancer_image, dancer_rgb, dancer_results = process_image(dancer_image_path)
        
        if not dancer_results.pose_landmarks:
            return {"error": "No pose detected in dancer image"}
        
        # Visualize the detected pose
        visualize_pose(dancer_rgb, dancer_results)
        
        # Compare with each reference pose in the sequence
        results = {}
        for pose_name in reference_sequence:
            results[pose_name] = self.compare_poses(
                dancer_results.pose_landmarks.landmark, 
                pose_name, 
                view
            )
        
        return results


In [8]:
# DANCE EVALUATOR CLASS
class DanceEvaluator:
    """Evaluates a complete dance performance and generates reports"""
    
    def __init__(self, analyzer):
        """Initialize the dance evaluator"""
        self.analyzer = analyzer
        
    def evaluate_performance(self, dancer_frontal_image, dancer_side_image, arm_sequence, foot_sequence):
        """Evaluate a dance performance from both frontal and side views"""
        # Analyze arm movements (frontal view)
        print("Analyzing arm movements (frontal view)...")
        arm_results = self.analyzer.analyze_dance_sequence(
            dancer_frontal_image, 
            arm_sequence, 
            view="front"
        )
        
        # Analyze footwork (side view)
        print("Analyzing footwork (side view)...")
        foot_results = self.analyzer.analyze_dance_sequence(
            dancer_side_image, 
            foot_sequence, 
            view="side"
        )
        
        # Combine results
        complete_evaluation = {
            "arm_movements": arm_results,
            "footwork": foot_results
        }
        
        return complete_evaluation
    
    def generate_feedback_report(self, evaluation_results):
        """Generate a comprehensive feedback report"""
        arm_results = evaluation_results.get("arm_movements", {})
        foot_results = evaluation_results.get("footwork", {})
        
        # Calculate overall performance score
        all_scores = []
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                all_scores.append(result["score"])
        
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                all_scores.append(result["score"])
        
        overall_score = np.mean(all_scores) if all_scores else 0
        
        # Generate HTML report
        html = f"""
        <h2>Spanish Dance Performance Evaluation</h2>
        <h3>Overall Score: {overall_score:.1f}%</h3>
        
        <h3>Arm Movements (Frontal View)</h3>
        <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr>
            <th style="padding: 8px; text-align: left;">Position</th>
            <th style="padding: 8px; text-align: left;">Score</th>
            <th style="padding: 8px; text-align: left;">Feedback</th>
        </tr>
        """
        
        # Add arm movement results
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                score = result["score"]
                message = result.get("message", "")
                
                # Determine rating color based on score
                if score >= 90:
                    color = "green"
                    rating = "Excellent"
                elif score >= 75:
                    color = "#5cb85c"  # lighter green
                    rating = "Good"
                elif score >= 60:
                    color = "orange"
                    rating = "Fair"
                else:
                    color = "red"
                    rating = "Needs Improvement"
                
                html += f"""
                <tr>
                    <td style="padding: 8px;">{pose_name}</td>
                    <td style="padding: 8px; color: {color};">{score:.1f}% ({rating})</td>
                    <td style="padding: 8px;">{message}</td>
                </tr>
                """
        
        html += """
        </table>
        
        <h3>Footwork (Side View)</h3>
        <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr>
            <th style="padding: 8px; text-align: left;">Position</th>
            <th style="padding: 8px; text-align: left;">Score</th>
            <th style="padding: 8px; text-align: left;">Feedback</th>
        </tr>
        """
        
        # Add footwork results
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                score = result["score"]
                message = result.get("message", "")
                
                # Determine rating color based on score
                if score >= 90:
                    color = "green"
                    rating = "Excellent"
                elif score >= 75:
                    color = "#5cb85c"  # lighter green
                    rating = "Good"
                elif score >= 60:
                    color = "orange"
                    rating = "Fair"
                else:
                    color = "red"
                    rating = "Needs Improvement"
                
                html += f"""
                <tr>
                    <td style="padding: 8px;">{pose_name}</td>
                    <td style="padding: 8px; color: {color};">{score:.1f}% ({rating})</td>
                    <td style="padding: 8px;">{message}</td>
                </tr>
                """
        
        html += """
        </table>
        
        <h3>Areas for Improvement</h3>
        <p>Focus on the following positions:</p>
        <ul>
        """
        
        # Find lowest-scoring positions for improvement focus
        all_results = []
        for pose_name, result in arm_results.items():
            if isinstance(result, dict) and "score" in result:
                all_results.append((pose_name, result["score"], "arm"))
        
        for pose_name, result in foot_results.items():
            if isinstance(result, dict) and "score" in result:
                all_results.append((pose_name, result["score"], "foot"))
        
        # Sort by score (ascending) and take the 3 lowest
        all_results.sort(key=lambda x: x[1])
        for pose_name, score, pose_type in all_results[:3]:
            html += f"<li><strong>{pose_name}</strong> ({pose_type} position): {score:.1f}%</li>"
        
        html += """
        </ul>
        
        <p>Practice these positions to improve your overall performance.</p>
        """
        
        # Display the HTML report
        display(HTML(html))
        
        return html
    
    def display_report(self, evaluation_results):
        """Display the feedback report"""
        return self.generate_feedback_report(evaluation_results)


In [9]:
# EXAMPLE USAGE
# Create the reference pose manager and add sample reference poses
def setup_reference_dataset():
    """Set up a sample reference dataset"""
    ref_manager = ReferencePoseManager(data_dir="spanish_dance_references")
    
    # Example: Add reference poses for arm movements (frontal view)
    arm_poses = {
        "brazos_en_quinta": {
            "path": "dataset/arms/quinta.jpg",
            "description": "Arms in fifth position, rounded above the head"
        },
        "braceo_flamenco": {
            "path": "dataset/arms/braceo.jpg",
            "description": "Flowing arm movement with wrists rotating"
        }
    }
    
    for name, data in arm_poses.items():
        ref_manager.add_reference_pose(name, data["path"], "arm", data["description"])
    
    # Example: Add reference poses for footwork (side view)
    foot_poses = {
        "planta_tacón": {
            "path": "dataset/feet/planta_tacon.jpg",
            "description": "Heel-to-toe rhythmic step"
        },
        "golpe": {
            "path": "dataset/feet/golpe.jpg",
            "description": "Sharp stamp of the whole foot"
        }
    }
    
    for name, data in foot_poses.items():
        ref_manager.add_reference_pose(name, data["path"], "foot", data["description"])
    
    return ref_manager

# Cell 8: Evaluate a dancer's performance
def evaluate_dancer_performance(dancer_name, frontal_image_path, side_image_path):
    """Evaluate a dancer's performance"""
    # Load the reference dataset
    ref_manager = ReferencePoseManager(data_dir="spanish_dance_references")
    ref_manager.load_references()
    
    # Create the analyzer and evaluator
    analyzer = PoseAnalyzer(ref_manager)
    evaluator = DanceEvaluator(analyzer)
    
    # Define the sequence of positions to evaluate
    arm_sequence = ["brazos_en_quinta", "braceo_flamenco"]
    foot_sequence = ["planta_tacón", "golpe"]
    
    # Evaluate the dancer's performance
    results = evaluator.evaluate_performance(
        frontal_image_path,
        side_image_path,
        arm_sequence,
        foot_sequence
    )
    
    # Display the evaluation report
    print(f"Performance Evaluation for {dancer_name}")
    evaluator.display_report(results)
    
    return results

# Cell 9: Run evaluation on test data
# Uncomment and run when ready
# setup_reference_dataset()
# evaluate_dancer_performance(
#     "Test Dancer",
#     "test_data/dancer_front.jpg",
#     "test_data/dancer_side.jpg"
# )


In [ ]:
#SPANISH DANCE POSE ANALYSIS TOOL

import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# GUI setup
root = tk.Tk()
root.title("Spanish Dance Pose Analysis Tool")
root.geometry("800x600")

# Function to load predefined images
def load_predefined_images():
    global predefined_images
    file_paths = filedialog.askopenfilenames(title="Select Predefined Position Images", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    predefined_images = [cv2.imread(file) for file in file_paths]
    messagebox.showinfo("Info", f"Loaded {len(predefined_images)} predefined images")

# Function to load exercise video
def load_exercise_video():
    global exercise_video_path
    exercise_video_path = filedialog.askopenfilename(title="Select Exercise Video", filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    messagebox.showinfo("Info", "Exercise video loaded successfully")

# Function to start real-time camera observation
def start_camera():
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process frame using MediaPipe Pose
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                h, w, _ = frame.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        
        cv2.imshow("Real-Time Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to generate checklist and report
def generate_report():
    # Placeholder logic for evaluation
    messagebox.showinfo("Report", "Checklist and report generated successfully!")

# GUI Buttons
btn_load_images = tk.Button(root, text="Load Predefined Images", command=load_predefined_images)
btn_load_images.pack(pady=10)

btn_load_video = tk.Button(root, text="Load Exercise Video", command=load_exercise_video)
btn_load_video.pack(pady=10)

btn_start_camera = tk.Button(root, text="Start Camera Observation", command=start_camera)
btn_start_camera.pack(pady=10)

btn_generate_report = tk.Button(root, text="Generate Report", command=generate_report)
btn_generate_report.pack(pady=10)

root.mainloop()

W0000 00:00:1741270696.943663 2573922 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1741270696.946025 2568791 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1741270696.984349 2573921 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741270697.000711 2573940 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741270697.015037 2573939 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-06 15:18:53.383 python[44073:2568791] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-06 15:18:53.383 python[44073:2568791] +[IMKInputSession subc